# Single Goal Environment

Here we want to train an agent to reach a goal using reinforcement learning algorithms.

## Warm-up
First, we do some installations.

In [ ]:
%tensorflow_version 1.x

!apt-get install graphviz libgraphviz-dev pkg-config
!apt-get install -y xvfb python-opengl ffmpeg
!pip install git+https://github.com/mhtb32/tl-env.git#egg=tl-env
!pip install stable-baselines==2.10.0 pyvirtualdisplay

Now we do imports and initializations.

In [ ]:
from pathlib import Path

import gym
from stable_baselines.sac.policies import MlpPolicy
from stable_baselines import SAC
from stable_baselines.common.callbacks import EvalCallback
from stable_baselines.common.evaluation import evaluate_policy

# noinspection PyUnresolvedReferences
import tl_env

Then, we specify a save path to save trained model and make the environments.

In [ ]:
(Path.cwd().parent / 'out').mkdir(exist_ok=True)
save_path = Path.cwd().parent / 'out'

env = gym.make('tl_env:SingleGoalIDM-v0')
eval_env = gym.make('tl_env:SingleGoalIDM-v0')

## Training
Now we train the agent using Soft Actor Critic(SAC) algorithm.

In [ ]:
eval_callback = EvalCallback(eval_env, eval_freq=3000, best_model_save_path=str(save_path))

model = SAC(MlpPolicy, env, verbose=1, buffer_size=10000)
model.learn(total_timesteps=40000, log_interval=200, callback=eval_callback)
model.save(str(save_path / 'final_model_40000'))
del model

## Testing
Now we test the agent for a few episodes to see how it is doing. We first define a simple helper function for
visualization of episodes:

In [ ]:
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
from gym.wrappers import Monitor
import base64
from tqdm.notebook import trange

display = Display(visible=0, size=(1400, 900))
display.start()

def show_video():
    html = []
    for mp4 in Path("../out/video").glob("*.mp4"):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append('''<video alt="{}" autoplay
                      loop controls style="height: 400px;">
                      <source src="data:../out/video/mp4;base64,{}" type="video/mp4" />
                 </video>'''.format(mp4, video_b64.decode('ascii')))
    # noinspection PyTypeChecker
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

Now we test the policy:

In [ ]:
env = Monitor(eval_env, '../out/video', force=True, video_callable=lambda episode: True)
model = SAC.load(str(save_path / 'best_model'))
for episode in trange(3, desc="Test episodes"):
    obs, done = env.reset(), False
    env.unwrapped.automatic_rendering_callback = env.video_recorder.capture_frame
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
env.close()
show_video()

## Evaluation

Finally, we evaluate the policy to have a quantitative sense of how it works.

In [ ]:
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10)

print(f"10-episode reward is {mean_reward:.2f} +/- {std_reward:.2f}")
